In [ ]:
!pip3 install datasets
!pip3 install tqdm
!pip3 install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd

from datasets import Dataset
from tqdm import tqdm

In [ ]:
import torch

if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

import os
import multiprocessing

num_cores = multiprocessing.cpu_count()

In [ ]:
def read_data(dataset='train'):
  def read_from_hf():
    import huggingface_hub
    huggingface_hub.login()

    from datasets import load_dataset
    dataset = load_dataset("aeirya/irhw3", use_auth_token=True)

    import pandas as pd
    text = dataset['train']['text']

    with open('buffer.txt', 'w') as file:
      file.write('\n'.join(text))

    df = pd.read_csv('buffer.txt', delimiter='\t')

    return df

  def read_from_drive():
    from google.colab import drive
    try:
      drive.mount('/content/drive')
    except:
      pass

    dir = '/content/drive/My Drive/Assignment/InformationRetrieval/hw3/dataset'
    file = f'{dir}/{dataset}_dataset.txt'

    return pd.read_csv(file, delimiter='\t')

  return read_from_drive()

df = read_data('train')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,user_id,item_id,rate,review_text
0,A2YKWYC3WQJX5J,B00106AC06,1,I usually love the Motions conditioners and ma...
1,A2LXC5ZHHP0WXP,B00AE07BMQ,1,Axe messy look styling gum is a product that w...
2,A3HLTHHLPKLRQA,B00AIQOKDY,1,I have always found liquid soap to be as much ...
3,A6N1DC5AMPLSK,B000F6RFX4,1,I've tried plenty of products that claim to he...
4,ALNFHVS3SC4FV,B0020122ZS,1,Suave Kids is one of my favorite brands of sha...


In [ ]:
# from data_reader import read_interaction_matrix as interactions, read_df

# R = interactions()
# df = read_df('train')
# df.head()

In [ ]:
users = list(set(df['user_id']))
items = list(set(df['item_id']))

n_users = len(users)
n_items = len(items)

user2idx = { x:i for i,x in enumerate(users) }
item2idx = { x:i for i,x in enumerate(items) }

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# model_name = 'bert-base-uncased'
# model_name = 'distilbert-base-uncased'
model_name = 'distilroberta-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

model.eval()
model.to(device)

In [ ]:
df = df.dropna()

tok_no_pad = tokenizer(df['review_text'].tolist(), padding=False, truncation=True)
n = [len(e.ids) for e in tok_no_pad[:]]
pd.Series(n).describe(percentiles=[0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
def tok(sentences, max_length=200):
    return tokenizer(
        sentences,
        add_special_tokens=True,
        return_tensors="pt",
        return_attention_mask=True,
        padding=True,
        truncation=True,
        max_length=max_length,
        # pad_to_max_length=True
    )


def bert(texts, tokenized=True):
    tok_input = tok(texts) if not tokenized else texts

    with torch.no_grad():
        output = model(**tok_input).last_hidden_state

    return output


def sent2vec(output, alpha=0.2):
    '''
    aggregate tokens in every sentence together

    alpha: the effect of max pooling

    @param output: (n_sentences x n_tokens x n_hidden_size)
    '''

    max_ = output.max(axis=1).values
    mean = output.mean(axis=1)
    return alpha * max_ + (1-alpha) * mean

In [ ]:
# from datasets import Dataset

# DF = df
# DF['n_tok'] = n
# DF = DF.dropna().sort_values(by='n_tok')[['item_id', 'review_text', 'n_tok']]
# ds = Dataset.from_pandas(DF.reset_index())

# def be(ds):
#     input = tokenize_ds(ds)
#     input.to(device)
#     out = bert(input)
#     input.to('cpu')
#     return {'out': out}

# enc_batch_size = 256
# enc_ds = ds.map(be, batched=True, batch_size=enc_batch_size)
# enc_ds.set_format('torch')

# groups = enc_ds.to_pandas().groupby('item_id')

# for item, group in groups:
#     out = torch.stack(group['out'].tolist())
#     print(out.shape)
#     break

In [ ]:
def tokenize_ds(ds):
    return tok(ds['review_text'])


enc = torch.zeros((len(items), 768))

DF = df.dropna().sort_values('item_id').reset_index()[['item_id', 'review_text']]

input_ds = Dataset.from_pandas(DF)\
  .map(tok, batched=True, batch_size=80000, input_columns='review_text', remove_columns=['review_text'])

input_ds.set_format('torch')

Map:   0%|          | 0/23037 [00:00<?, ? examples/s]

In [ ]:
input_ds

Dataset({
    features: ['item_id', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 23037
})

In [ ]:
groups = DF.groupby(by='item_id', dropna=True)

for item, group in tqdm(groups):

    input = input_ds[group.index]
    input = {k:input[k].to(device) for k in ['input_ids', 'token_type_ids', 'attention_mask'] if k in input}

    out = bert(input)

    # average of all sentences
    v = sent2vec(out).mean(axis=0)

    enc[item2idx[item], :] = v

100%|██████████| 733/733 [04:07<00:00,  2.96it/s]


In [ ]:
def encode_items_bert(ds):

In [ ]:
from sklearn.decomposition import PCA

res = PCA(64).fit_transform(enc)